In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 15 20:56:04 2018

@author: corey


Cleaning up the  Yelp Dataset

"""
import pandas as pd
from pprint import pprint
import numpy as np
import json
import censusgeocode as cg

df = pd.read_csv('LA_County_restaurants.csv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')
df.head()

df = df[df['city'] == "Los Angeles"]

,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,...,review_count,transactions,url,Latitude,Longitude,city,zipcode,category1,category2,category3
35,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.0047569274902, 'longitude': -1...",(323) 561-1675,54768.500159,fettes-schwein-food-truck-los-angeles-3,https://s3-media3.fl.yelpcdn.com/bphoto/3LOYvP...,False,"{'address1': '', 'address2': '', 'address3': '...",Fettes Schwein Food Truck,1.323561e+10,...,14,[],https://www.yelp.com/biz/fettes-schwein-food-t...,-118.259079,34.004757,Los Angeles,90011,Food Trucks,Hot Dogs,Burgers
45,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.10161, 'longitude': -118.30206}",(323) 477-6764,53442.069549,the-tropic-truck-los-angeles-5,https://s3-media2.fl.yelpcdn.com/bphoto/Fq9EuT...,False,"{'address1': None, 'address2': None, 'address3...",The Tropic Truck,1.323478e+10,...,50,[],https://www.yelp.com/biz/the-tropic-truck-los-...,-118.302060,34.101610,Los Angeles,90027,Food Trucks,Caribbean,Caterers
47,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.1463062227897, 'longitude': -1...",(818) 373-9767,64863.573440,okamoto-kitchen-los-angeles-2,https://s3-media3.fl.yelpcdn.com/bphoto/9bs7YT...,False,"{'address1': None, 'address2': None, 'address3...",Okamoto Kitchen,1.818374e+10,...,168,"['delivery', 'pickup', 'restaurant_reservation']",https://www.yelp.com/biz/okamoto-kitchen-los-a...,-118.425397,34.146306,Los Angeles,91423,Food Trucks,Japanese,Caterers
53,"[{'alias': 'filipino', 'title': 'Filipino'}, {...","{'latitude': 34.0569114685059, 'longitude': -1...",(888) 907-2248,50926.777789,white-rabbit-truck-los-angeles-4,https://s3-media1.fl.yelpcdn.com/bphoto/-_twM9...,False,"{'address1': None, 'address2': None, 'address3...",White Rabbit Truck,1.888907e+10,...,569,[],https://www.yelp.com/biz/white-rabbit-truck-lo...,-118.243950,34.056911,Los Angeles,90012,Filipino,Food Trucks,Caterers
72,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.21332, 'longitude': -118.35861}",(818) 282-4301,58069.947811,shawarma-and-falafel-palace-food-truck-los-ang...,https://s3-media4.fl.yelpcdn.com/bphoto/jKyVhe...,False,"{'address1': '7782 San Fernando Rd', 'address2...",Shawarma and Falafel Palace Food Truck,1.818282e+10,...,1,[],https://www.yelp.com/biz/shawarma-and-falafel-...,-118.358610,34.213320,Los Angeles,91352,Food Trucks,Middle Eastern,NaN
964,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.029109954834, 'longitude': -11...",(562) 794-6484,51357.806229,urban-espresso-los-angeles-6,https://s3-media3.fl.yelpcdn.com/bphoto/bUKrBN...,False,"{'address1': '', 'address2': None, 'address3':...",Urban Espresso,1.562795e+10,...,25,[],https://www.yelp.com/biz/urban-espresso-los-an...,-118.283066,34.029110,Los Angeles,90007,Food Trucks,Coffee & Tea,Breakfast & Brunch
1473,"[{'alias': 'french', 'title': 'French'}]","{'latitude': 34.13904, 'longitude': -118.20416}",(323) 255-5111,12323.773752,cafe-beaujolais-los-angeles,https://s3-media2.fl.yelpcdn.com/bphoto/MZ0MtL...,False,"{'address1': '1712 Colorado Blvd', 'address2':...",Cafe Beaujolais,1.323256e+10,...,1261,[],https://www.yelp.com/biz/cafe-beaujolais-los-a...,-118.204160,34.139040,Los Angeles,90041,French,NaN,NaN
1480,"[{'alias': 'breakfast_brunch', 'title': 'Break...","{'latitude': 34.12768, 'longitude': -118.21722}",(323) 474-6420,14061.604868,rockn-egg-cafe-los-angeles-3,https://s3-media2.fl.yelpcdn.com/bphoto/YwVMqf...,False,"{'address1': '4616 Eagle Rock Blvd', 'address2...",Rock'n Egg Cafe,1.323475e+10,...,33,[],https://www.yelp.com/biz/rockn-egg-cafe-los-an...,-118.217220,34.127680,Los Angeles,90041,Breakfast & Brunch,Diners,Sandwiches
1487,"[{'alias': 'vegan', 'title': 'Vegan'}]","{'latitude': 34.1343840632986, 'longitude': -1...",(323) 775-6614,13367.367100,the-vegan-hooligans-los-angeles,https://s3-media1.fl.yelpcdn.com/bphoto/cXOsCI...,False,"{'address1': '4862 Eagle Rock B

In [100]:
# this variable is for testing extract_location
loc_sample = "{'address1': '349 N State College Blvd', 'address2': None, 'address3': None, 'city': 'Fullerton', 'zip_code': '92831', 'country': 'US', 'state': 'CA', 'display_address': ['349 N State College Blvd', 'Fullerton, CA 92831']}"


def extract_category(df):
    category1 = []
    category2 = []
    category3 = []
    for x in df['categories']:
        json_rec = json.loads(x.replace("'", '"'))
        category1.append(json_rec[0]['title'])
        try:
            category2.append(json_rec[1]['title'])
        except:
            category2.append('')
        try:
            category3.append(json_rec[2]['title'])
        except:
            category3.append('')
    df['category1'] = category1
    df['category2'] = category2
    df['category3'] = category3

extract_category(df)

In [87]:
#pprint(loc_sample)


def extract_location(df):
    '''
    breaks yelp's location data along the key that we want and then
    returns it as a new dataframe
    '''
    cities = []
    zip_codes = []
    streets = []
    #print(df['location'])
    #print(json.dumps(df['location']))
    placeholder = []
    for row in df['location']:
        try:
            address = row.split("'display_address': ['")[1].rstrip("']}").split("', '")
            city = address[1].split(",")[0]
            #print(city)
            cities.append(city)
            street = address[0]
            print(street, city)
            streets.append(address[0])
            #print(address)
        except:
            print('error: \n')
            print(row)
            cities.append('none')
            streets.append('none')
        
        try:
            zip_code = row.split("'zip_code': ")[1].split(",")[0].strip("'")
        except:
            zip_code = np.nan
            
        zip_codes.append(zip_code)
    df['city'] = cities
    df['street'] = streets
    df['zipcode'] = zip_codes
        

       
extract_location(df)    

3165 Yorba Linda Blvd Fullerton
349 N State College Blvd Fullerton
660 N Rose Dr Placentia
2931 Nutwood Ave Fullerton
1325 E Chapman Ave Fullerton
711 N Placentia Ave Fullerton
3203 N Glassell St Orange
114 W Commonwealth Ave Fullerton
305 N State College Blvd Fullerton
2323 E Chapman Ave Ste B
512 N State College Blvd Fullerton
2905 E Miraloma Ave Ste 3
616 W Chapman Ave Placentia
3405 E Imperial Hwy Brea
305 N State College Blvd Fullerton
201 S Bradford Ave Placentia
2279 N Tustin St Orange
1956 N Placentia Ave Fullerton
440 S Anaheim Blvd Ste 104
733 N Placentia Ave Fullerton
1450 N Kraemer Blvd Placentia
1060 E Imperial Hwy Brea
141 E Commonwealth Ave Fullerton
202 W Lincoln Ave Anaheim
2445 E Imperial Hwy Ste M
229 E Commonwealth Ave Fullerton
2555 E Chapman Ave Fullerton
1071 N Tustin Ave Ste 110
2445 E Imperial Hwy Ste F
401 S Magnolia Ave Anaheim
3458 E Orangethorpe Ave Anaheim
1343 E Chapman Ave Fullerton
907 E Yorba Linda Blvd Placentia
374 S Main St Orange
184 S Harbor Blvd 

1313 S Harbor Blvd Anaheim
26676 Portola Pkwy Ste D
2136 Colorado Blvd Los Angeles
440 S Anaheim Blvd No 209-B
10708 Los Alamitos Blvd Los Alamitos
Buena Vista St Anaheim
121 Mc Fadden Pl Newport Beach
13221 Garden Grove Blvd Garden Grove
14258 Newport Ave Tustin
10043 Artesia Pl Bellflower
6740 Greenleaf Ave Whittier
12681 Harbor Blvd Garden Grove
619 N Azusa Ave Azusa
3757 Riverside Dr Ste A
400 W Disney Way Ste 337
46 E Huntington Dr Arcadia
1834 Puente Ave Baldwin Park
7511 Edinger Ave Huntington Beach
228 W 2nd St Pomona
129 E Arrow Hwy Glendora
11332 Washington Blvd Whittier
6339 Etiwanda Ave Mira Loma
5363 E Arrow Hwy Montclair
2114 E Lincoln Ave Anaheim
5555 Hamner Ave Norco
417 S Brookhurst St Anaheim
53 E Union St Pasadena
30 E Orangethorpe Ave Anaheim
10602 Lower Azusa Rd El Monte
1822 Chico Ave South El Monte
2000 E Gene Autry Way Anaheim
11761 Edinger Ave Fountain Valley
820 W 19th St Costa Mesa
16701 Pacific Coast Hwy Sunset Beach
1060 Walnut Ave Tustin
16025 Gale Ave Ind

814 W 19th St Costa Mesa
4523 Gage Ave Bell
3583 N Los Coyotes Diagonal Long Beach
2839 S Diamond Bar Blvd Diamond Bar
1821 W Commonwealth Ave Ste H
194 Marina Dr Ste 101
5621 N Figueroa St Los Angeles
116 E Chapman Ave Orange
4100 Donald Douglas Dr Long Beach
11001 S Central Ave Ontario
9469 Foothill Blvd Rancho Cucamonga
2438 E Katella Ave Anaheim
11265 Topaz Ave Fountain Valley
22722 Ctr Dr Ste A
5100 E 2nd St Long Beach
2493 Park Ave Ste 41
14001 Newport Ave Tustin
1542 W Holt Blvd Ontario
101 Main St Seal Beach
2001 Westcliff Dr Ste 100
error: 

{'address1': '1776 S Disneyland Dr', 'address2': '', 'address3': "Disney's California Adventure", 'city': 'Anaheim', 'zip_code': '92802', 'country': 'US', 'state': 'CA', 'display_address': ['1776 S Disneyland Dr', "Disney's California Adventure", 'Anaheim, CA 92802']}
8228 Firestone Blvd Downey
2142 E 4th St Long Beach
2421 E Artesia Blvd Long Beach
2471 Whittier Blvd Los Angeles
28 W Main St Alhambra
8082 Adams Ave Huntington Beach
4702 W

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11684 entries, 0 to 11683
Data columns (total 22 columns):
categories       11684 non-null object
coordinates      11684 non-null object
display_phone    11338 non-null object
distance         11684 non-null float64
id               11684 non-null object
image_url        11575 non-null object
is_closed        11684 non-null bool
location         11684 non-null object
name             11684 non-null object
phone            11338 non-null float64
price            11192 non-null object
rating           11684 non-null float64
review_count     11684 non-null int64
transactions     11684 non-null object
url              11684 non-null object
Latitude         11684 non-null float64
Longitude        11684 non-null float64
city             11684 non-null object
zipcode          11663 non-null object
category1        11684 non-null object
category2        11684 non-null object
category3        11684 non-null object
dtypes: bool(1), float64(5), in

In [120]:
#Get census tracts
from LA_zipcodes import zip_codes
results = []
for index,row in df.iterrows():
    
        


checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91773
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91773
checking for: 

91750
checking for: 

91750
checking for: 

91773
checking for: 

91711
checking for: 

91750
checking for: 

91750
checking for: 

91773
checking for: 

91750
checking for: 

90249
checking for: 

90011
checking for: 

91711
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

90027
checking for: 

91423
checking for: 

91750
checking for: 

90012
checking for: 

91711
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking for: 

91773
checking for: 

91750
checking for: 

91750
checking for: 

91750
checking f

91775
checking for: 

91030
checking for: 

90041
checking for: 

90042
checking for: 

91103
checking for: 

91775
checking for: 

91107
checking for: 

91101
checking for: 

90041
checking for: 

91030
checking for: 

91208
checking for: 

91107
checking for: 

91105
checking for: 

91101
checking for: 

91104
checking for: 

90042
checking for: 

91101
checking for: 

91007
checking for: 

90041
checking for: 

91007
checking for: 

90065
checking for: 

90041
checking for: 

91104
checking for: 

91106
checking for: 

91007
checking for: 

90042
checking for: 

91104
checking for: 

91103
checking for: 

91103
checking for: 

91107
checking for: 

91103
checking for: 

91352
checking for: 

91104
checking for: 

91107
checking for: 

91107
checking for: 

91030
checking for: 

91105
checking for: 

91108
checking for: 

91105
checking for: 

91030
checking for: 

90041
checking for: 

91107
checking for: 

91105
checking for: 

91007
checking for: 

91104
checking for: 

90042
chec


90660
checking for: 

90640
checking for: 

90042
checking for: 

91770
checking for: 

91801
checking for: 

91801
checking for: 

91776
checking for: 

90640
checking for: 

90270
checking for: 

91731
checking for: 

90023
checking for: 

90033
checking for: 

90270
checking for: 

90601
checking for: 

90063
checking for: 

90023
checking for: 

91755
checking for: 

90031
checking for: 

91755
checking for: 

91770
checking for: 

90640
checking for: 

90602
checking for: 

90280
checking for: 

91754
checking for: 

91803
checking for: 

90270
checking for: 

90640
checking for: 

90063
checking for: 

90029
checking for: 

90063
checking for: 

91754
checking for: 

90022
checking for: 

90022
checking for: 

91770
checking for: 

91776
checking for: 

90023
checking for: 

90031
checking for: 

91770
checking for: 

90241
checking for: 

91780
checking for: 

90601
checking for: 

91801
checking for: 

90033
checking for: 

91803
checking for: 

90280
checking for: 

91733
che

checking for: 

90713
checking for: 

90806
checking for: 

90808
checking for: 

90723
checking for: 

90802
checking for: 

90712
checking for: 

90815
checking for: 

90650
checking for: 

90814
checking for: 

90805
checking for: 

90803
checking for: 

90221
checking for: 

90701
checking for: 

90802
checking for: 

90802
checking for: 

90706
checking for: 

90813
checking for: 

90815
checking for: 

90810
checking for: 

90803
checking for: 

90802
checking for: 

90815
checking for: 

90802
checking for: 

90803
checking for: 

90804
checking for: 

90803
checking for: 

90808
checking for: 

90803
checking for: 

90804
checking for: 

90701
checking for: 

90701
checking for: 

90807
checking for: 

90703
checking for: 

90813
checking for: 

90701
checking for: 

90807
checking for: 

90706
checking for: 

90804
checking for: 

90703
checking for: 

90806
checking for: 

90712
checking for: 

90712
checking for: 

90803
checking for: 

90706
checking for: 

90806
checking f

91356
checking for: 

91411
checking for: 

91403
checking for: 

91303
checking for: 

91340
checking for: 

91324
checking for: 

91324
checking for: 

91364
checking for: 

91340
checking for: 

91335
checking for: 

91326
checking for: 

91342
checking for: 

91316
checking for: 

91335
checking for: 

91402
checking for: 

91303
checking for: 

91324
checking for: 

91335
checking for: 

91304
checking for: 

91304
checking for: 

91436
checking for: 

91311
checking for: 

91306
checking for: 

91306
checking for: 

91403
checking for: 

91343
checking for: 

91344
checking for: 

91331
checking for: 

91325
checking for: 

91324
checking for: 

91343
checking for: 

91335
checking for: 

91324
checking for: 

91343
checking for: 

91364
checking for: 

91403
checking for: 

91325
checking for: 

91340
checking for: 

91367
checking for: 

91343
checking for: 

91335
checking for: 

91405
checking for: 

91406
checking for: 

91303
checking for: 

91364
checking for: 

91311
chec

...

91744
checking for: 

91791
checking for: 

91746
checking for: 

91006
checking for: 

91016
checking for: 

91016
checking for: 

91744
checking for: 

91016
checking for: 

91016
checking for: 

91732
checking for: 

91010
checking for: 

91010
checking for: 

91702
checking for: 

91723
checking for: 

91744
checking for: 

91746
checking for: 

91740
checking for: 

91790
checking for: 

91791
checking for: 

91016
checking for: 

91016
checking for: 

91740
checking for: 

91723
checking for: 

91744
checking for: 

91006
checking for: 

91006
checking for: 

91740
checking for: 

91791
checking for: 

91732
checking for: 

91723
checking for: 

91706
checking for: 

91016
checking for: 

91723
checking for: 

91010
checking for: 

91706
checking for: 

91016
checking for: 

91744
checking for: 

91744
checking for: 

91706
checking for: 

91791
checking for: 

91016
checking for: 

91016
checking for: 

91016
checking for: 

91706
checking for: 

91790
checking for: 

91791
chec

checking for: 

91214
checking for: 

91342
checking for: 

91331
checking for: 

91331
checking for: 

91331
checking for: 

91042
checking for: 

91331
checking for: 

91331
checking for: 

91331
checking for: 

91331
checking for: 

91331
checking for: 

91352
checking for: 

91352
checking for: 

91331
checking for: 

91352
checking for: 

91352
checking for: 

91352
checking for: 

91342
checking for: 

91331
checking for: 

91040
checking for: 

91340
checking for: 

91352
checking for: 

91605
checking for: 

91214
checking for: 

91214
checking for: 

91352
checking for: 

91331
checking for: 

91605
checking for: 

91214
checking for: 

91331
checking for: 

91352
checking for: 

91042
checking for: 

91342
checking for: 

91040
checking for: 

91042
checking for: 

91352
checking for: 

91331
checking for: 

91331
checking for: 

91605
checking for: 

91342
checking for: 

91351
checking for: 

91214
checking for: 

91352
checking for: 

91387
checking for: 

91331
checking f

91505
checking for: 

90016
checking for: 

90028
checking for: 

90046
checking for: 

90027
checking for: 

90068
checking for: 

90006
checking for: 

90028
checking for: 

90006
checking for: 

91204
checking for: 

90035
checking for: 

90038
checking for: 

90010
checking for: 

90020
checking for: 

90027
checking for: 

90068
checking for: 

91204
checking for: 

90027
checking for: 

90004
checking for: 

91352
checking for: 

91505
checking for: 

90026
checking for: 

90020
checking for: 

90036
checking for: 

90057
checking for: 

91606
checking for: 

90010
checking for: 

91206
checking for: 

90004
checking for: 

91604
checking for: 

90028
checking for: 

91502
checking for: 

91505
checking for: 

91601
checking for: 

90069
checking for: 

90048
checking for: 

90048
checking for: 

90046
checking for: 

90020
checking for: 

90210
checking for: 

90006
checking for: 

90013
checking for: 

90029
checking for: 

91204
checking for: 

91607
checking for: 

90068
chec

90013
checking for: 

90249
checking for: 

90248
checking for: 

90254
checking for: 

90278
checking for: 

90301
checking for: 

90266
checking for: 

90003
checking for: 

90266
checking for: 

90504
checking for: 

90277
checking for: 

90250
checking for: 

90277
checking for: 

90249
checking for: 

90250
checking for: 

90008
checking for: 

90250
checking for: 

90007
checking for: 

90302
checking for: 

90066
checking for: 

90007
checking for: 

90301
checking for: 

90047
checking for: 

90502
checking for: 

90094
checking for: 

90013
checking for: 

90745
checking for: 

90220
checking for: 

90301
checking for: 

90505
checking for: 

90061
checking for: 

90249
checking for: 

90277
checking for: 

90014
checking for: 

90277
checking for: 

90277
checking for: 

90045
checking for: 

90001
checking for: 

90006
checking for: 

90230
checking for: 

90245
checking for: 

90034
checking for: 

90230
checking for: 

90013
checking for: 

90505
checking for: 

90250
chec

checking for: 

90277
checking for: 

90277
checking for: 

90745
checking for: 

90745
checking for: 

90731
checking for: 

90745
checking for: 

90503
checking for: 

90277
checking for: 

90503
checking for: 

90505
checking for: 

90746
checking for: 

90505
checking for: 

90277
checking for: 

90731
checking for: 

90744
checking for: 

90744
checking for: 

90746
checking for: 

90503
checking for: 

90732
checking for: 

90501
checking for: 

90710
checking for: 

90505
checking for: 

90277
checking for: 

90744
checking for: 

90746
checking for: 

90501
checking for: 

90277
checking for: 

90744
checking for: 

90501
checking for: 

90277
checking for: 

90503
checking for: 

90744
checking for: 

90505
checking for: 

90277
checking for: 

90746
checking for: 

90501
checking for: 

90277
checking for: 

90503
checking for: 

90731
checking for: 

90745
checking for: 

90503
checking for: 

90745
checking for: 

90505
checking for: 

90505
checking for: 

90501
checking f

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

In [109]:
result = cg.onelineaddress('1600 Pennsylvania Avenue, Washington, DC')


In [110]:
result[0]

{'addressComponents': {'city': 'WASHINGTON',
  'fromAddress': '1600',
  'preDirection': '',
  'preQualifier': '',
  'preType': '',
  'state': 'DC',
  'streetName': 'PENNSYLVANIA',
  'suffixDirection': 'NW',
  'suffixQualifier': '',
  'suffixType': 'AVE',
  'toAddress': '1698',
  'zip': '20502'},
 'coordinates': {'x': -77.03535, 'y': 38.898754},
 'geographies': {'2010 Census Blocks': [{'AREALAND': 91475,
    'AREAWATER': 0,
    'BASENAME': '1031',
    'BLKGRP': '1',
    'BLOCK': '1031',
    'CENTLAT': '+38.8971157',
    'CENTLON': '-077.0365336',
    'COUNTY': '001',
    'FUNCSTAT': 'S',
    'GEOID': '110010062021031',
    'INTPTLAT': '+38.8971157',
    'INTPTLON': '-077.0365336',
    'LSADC': 'BK',
    'LWBLKTYP': 'L',
    'MTFCC': 'G5040',
    'NAME': 'Block 1031',
    'OBJECTID': 1760924,
    'OID': 210403964788146,
    'STATE': '11',
    'SUFFIX': '',
    'TRACT': '006202'}],
  'Census Tracts': [{'AREALAND': 6539774,
    'AREAWATER': 4970897,
    'BASENAME': '62.02',
    'CENTLAT': 